In [38]:
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import pandas as pd
from mrjob.job import MRJob


In [40]:
data=pd.read_csv("10k_view_data.csv")

Pandas check for the result

In [41]:
data_even=data[data["station_num"]%2==0]
mask=(data_even["event_time"]>=200000) &( data_even["event_time"]<=225959)
data_prime=data_even[mask]
counts=data_prime["prog_code"].value_counts()
print(counts[0],counts.keys()[0])

181 SP003189730000


In [42]:
%%file result.py

from mrjob.job import MRJob
from mrjob.step import MRStep


class MRMaxEvenPrimeTimeShow(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_values,
                   combiner=self.combiner_count_values,
                   reducer=self.reducer_count_values),
            MRStep(reducer=self.reducer_find_max)
        ]

    def mapper_get_values(self, _, line):
        if line.startswith('mso_code'):
            return
        fields =line.split(',')
        if ((int(fields[4])%2 == 0) and (int(fields[3]) >=200000 )and (int(fields[3]) <=225959)):
            yield  str(fields[5]),1
        


    def combiner_count_values(self,fields,counts):
        yield fields, sum(counts)


    def reducer_count_values(self ,fields, counts):
        yield None, (sum(counts), fields)


    def reducer_find_max(self, _, fields_count_pairs):
        yield max(fields_count_pairs)



# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    MRMaxEvenPrimeTimeShow.run()


Overwriting result.py


In [43]:
! python result.py  10k_view_data.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/result.root.20230412.084606.088229
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/result.root.20230412.084606.088229/output
Streaming final output from /tmp/result.root.20230412.084606.088229/output...
181	"SP003189730000"
Removing temp directory /tmp/result.root.20230412.084606.088229...


I got the same answer ☺